# **Requirement**

# Import Library

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import nltk
import os
import re
import string
import tokenizer
import gensim
import warnings 
import sklearn
import multiprocessing
import tensorflow as tf
import time

from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from kbbi import KBBI

Using TensorFlow backend.


In [2]:
import pandas
import numpy
import sqlite3
import nltk
from os import path

In [3]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

# **Read Data**

In [4]:
#Melakukan connection terhadap database
connection = sqlite3.connect(path.join("D:/","KULIAH", "Semester 8", "my-sems-8", "ML","DBnews.db"))

In [5]:
#Mengambil comments dari database
df_tbk = pandas.read_sql('select * from "t_comments"',connection)
connection.close()

In [8]:
#Database bahasa yang baku 
connection = sqlite3.connect(path.join("D:/","KULIAH", "Semester 8", "my-sems-8", "ML","korpus.db"))
df_cwords = pd.read_sql_query("SELECT * from korpus_kata", connection)
df_csentence = pd.read_sql_query("SELECT * from korpus_kalimat", connection)

In [9]:
df_cwords

,id,kata,tag
0,1,yang,p
1,2,untuk,p
2,3,dengan,p
3,4,dapat,adv
4,5,atau,p
5,6,mg,n
6,7,tidak,adv
7,8,digunakan,Entri tidak ditemukan
8,9,dokter,n
9,10,dari,p


In [10]:
df_csentence_kalimat = list(df_csentence['kalimat'])

In [11]:
df_cwords_kata = list(df_cwords['kata'])

In [12]:
df_cwords_kata

['yang',
 'untuk',
 'dengan',
 'dapat',
 'atau',
 'mg',
 'tidak',
 'digunakan',
 'dokter',
 'dari',
 'obat',
 'ini',
 'dalam',
 'terjadi',
 'penggunaan',
 'merupakan',
 'membantu',
 'juga',
 'infeksi',
 'adalah',
 'efek',
 'samping',
 'darah',
 'seperti',
 'penyakit',
 'tubuh',
 'kulit',
 'gejala',
 'lebih',
 'oleh',
 'jika',
 'harus',
 'lain',
 'sebagai',
 'dosis',
 'mengandung',
 'pemeriksaan',
 'dilakukan',
 'diminum',
 'sakit',
 'karena',
 'makan',
 'nyeri',
 'bisa',
 'kondisi',
 'menyebabkan',
 'mungkin',
 'gangguan',
 'anak',
 'beberapa',
 'itu',
 'bila',
 'sehingga',
 'selama',
 'tersebut',
 'segera',
 'namun',
 'pemakaian',
 'memiliki',
 'antara',
 'secara',
 'alergi',
 'mengalami',
 'orang',
 'mengobati',
 'selain',
 'jantung',
 'bakteri',
 'setiap',
 'hari',
 'rasa',
 'kesehatan',
 'makanan',
 'tahun',
 'bayi',
 'mata',
 'hamil',
 'pengobatan',
 'kali',
 'terhadap',
 'sebelum',
 'tinggi',
 'penderita',
 'disebabkan',
 'menjadi',
 'adanya',
 'ibu',
 'tablet',
 'mencegah',
 'he

# **Preprocessing Data**
Lowercase, Tokenizing, and Punctuation

In [13]:
df_tbk['comment']

0         WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KA...
1         Mantab.. ini br dakwah yg sejati.. kl d tempat...
2                                     Salut buat Gus Miftah
3         yg nyinyir gak pernah lihat dan baca atw menge...
4         Ada adabnya sholawat. Lebih baik ajak ke majel...
5         Setiap ulama punya jalan dan cara dakwah masin...
6                                   Umpanin nocannya yak...
7                            Itulah ceramah yg sebenarnya. 
8                                   Besok bugil aja selawat
9         Bedanya Gus Miftah dengan FPI  / HTI bagaikan ...
10        Knp di tempat kerjanya? Mending diundang ke ma...
11        nah ini ulama beneran...diterima di semua kala...
12        Kerennn.... saya pernah dpt info seperti ini d...
13        Hebat Gus.Lanjutkan!inilah makna dakwah sebena...
14              Diskotik syariah jadunya nih wkwkwk..mantap
15        Niat dan tujuan yg baik pasti mendapat berkah ...
16        mantaaapppp..buat di gedung DP

In [14]:
#lowercase kolom komentar b.tdk baku
df_tbk['comment'] = df_tbk['comment'].str.lower() 

In [15]:
#Tokenizing kolom komentar b.tdk baku
df_tbk['comment'][0].split

<function str.split(sep=None, maxsplit=-1)>

In [16]:
#Punctuation kolom komentar b.tdk baku
#df_tbk['comment'] = df_tbk['comment'].str.replace('[^\w\s]', '')
for i in range(0, len(df_tbk_comment)):
    comments = df_tbk_comment[i]
    for j in range(0, len(comments)):
        sub_comment = df_tbk_comment[i][j]
        for k in sub_comment:
            if k in string.punctuation:
                df_tbk_comment[i][j] = df_tbk_comment[i][j].replace(k, "")


In [224]:
#membersihkan karakter yang tidak diperlukan 
invalid = '�'
for i in range(0, len(df_tbk_comment)):
    comment = df_tbk_comment[i]
    for j in range(0, len(comment)):
        n_comments = df_tbk_comment[i][j]
        for k in n_comments:
            if k in invalid:
                df_tbk_comment[i][j] = df_tbk_comment[i][j].replace(k, "")

In [225]:
df_tbk['comment'][0]
print((df_tbk['comment'][0]).split())

In [226]:
#Menghapus yg kosong
for comment in df_tbk_comment:
    for sub_comment in comment:
        if sub_comment == '':
            comment.remove(sub_comment)

In [18]:
df_tbk_comment = []
for i in range(598097):{
  (df_tbk_comment.append((df_tbk['comment'][i]).split()))
}

In [23]:
df_cwords_kata

['yang',
 'untuk',
 'dengan',
 'dapat',
 'atau',
 'mg',
 'tidak',
 'digunakan',
 'dokter',
 'dari',
 'obat',
 'ini',
 'dalam',
 'terjadi',
 'penggunaan',
 'merupakan',
 'membantu',
 'juga',
 'infeksi',
 'adalah',
 'efek',
 'samping',
 'darah',
 'seperti',
 'penyakit',
 'tubuh',
 'kulit',
 'gejala',
 'lebih',
 'oleh',
 'jika',
 'harus',
 'lain',
 'sebagai',
 'dosis',
 'mengandung',
 'pemeriksaan',
 'dilakukan',
 'diminum',
 'sakit',
 'karena',
 'makan',
 'nyeri',
 'bisa',
 'kondisi',
 'menyebabkan',
 'mungkin',
 'gangguan',
 'anak',
 'beberapa',
 'itu',
 'bila',
 'sehingga',
 'selama',
 'tersebut',
 'segera',
 'namun',
 'pemakaian',
 'memiliki',
 'antara',
 'secara',
 'alergi',
 'mengalami',
 'orang',
 'mengobati',
 'selain',
 'jantung',
 'bakteri',
 'setiap',
 'hari',
 'rasa',
 'kesehatan',
 'makanan',
 'tahun',
 'bayi',
 'mata',
 'hamil',
 'pengobatan',
 'kali',
 'terhadap',
 'sebelum',
 'tinggi',
 'penderita',
 'disebabkan',
 'menjadi',
 'adanya',
 'ibu',
 'tablet',
 'mencegah',
 'he

# **CREATE WORD2VEC MODEL**


untuk bagian komentar (bhs tidak baku)

In [24]:
df_tbk_comment

[['wkwkwk',
  'gak',
  'sekalian',
  'pak',
  'selawat',
  'ke',
  'dpr',
  'kan',
  'banyak',
  'koruptor',
  'disana'],
 ['mantab',
  'ini',
  'br',
  'dakwah',
  'yg',
  'sejati',
  'kl',
  'd',
  'tempat',
  'umum',
  'mah',
  'dh',
  'biasa',
  'ini',
  'luar',
  'biasa'],
 ['salut', 'buat', 'gus', 'miftah'],
 ['yg',
  'nyinyir',
  'gak',
  'pernah',
  'lihat',
  'dan',
  'baca',
  'atw',
  'mengetahui',
  'sejarah',
  'dakwah',
  'walisongo',
  'sunan',
  'kali',
  'jagasmua',
  'ada',
  'cara',
  'demi',
  'kebaikan',
  'asal',
  'jng',
  'ajk',
  'dg',
  'cara',
  'berjinah',
  'lakukan',
  'dg',
  'porsi',
  'dan',
  'sikon',
  'yg',
  'tepat'],
 ['ada',
  'adabnya',
  'sholawat',
  'lebih',
  'baik',
  'ajak',
  'ke',
  'majelis',
  'ilmu'],
 ['setiap',
  'ulama',
  'punya',
  'jalan',
  'dan',
  'cara',
  'dakwah',
  'masing2',
  'jika',
  'kamu',
  'melihat',
  'kemunkaran',
  'lawan',
  'dengan',
  'tanganmukekuatanmu',
  'jika',
  'tidak',
  'mampu',
  'lawan',
  'dengan'

In [25]:
# Create comment CBOW model 
#window (default 5) - jarak maksimum antara target word dan kata diantara target word
#size(default 100) - dimensi dari embedding, misalnya panjang dari kepadatan vektor untuk merepresentasikan tiap token (per kata)
#min_count (default 5) - minimum hitungan kata yang dipertimbangkan ketika model di train; kata dengan kejadian lebih sedikit dari min_count akan diabaikan
#workers (default 3) - the number of partitions during training 
model_comment_cbow = gensim.models.Word2Vec(df_tbk_comment, min_count = 1,  
                              size = 100, window = 5, workers = 3, sg=0, iter= 10) 

In [26]:
model_comment_cbow['merasakan']

C:\Users\Rumondang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 5.246871  ,  1.747587  ,  2.1439085 , -0.93315357,  0.38337466,
       -0.95463765,  1.3547902 , -0.796875  , -2.331518  , -0.69903713,
       -0.12209201,  1.3976669 ,  0.1050434 ,  1.9369282 , -0.41857636,
        0.49162745,  3.233869  ,  2.714414  , -1.1257656 ,  0.34337994,
        3.4669893 ,  2.3339589 ,  1.9986745 , -1.0852913 , -0.9851202 ,
        2.0615256 , -0.23509894, -0.21009617, -1.0737959 , -0.81477183,
        3.5252295 ,  0.2784295 , -0.38722897,  1.7935803 , -1.1299781 ,
        0.3858605 , -2.354193  , -0.6816008 , -0.02073952, -2.1683092 ,
        2.8286402 , -0.82090575,  3.306741  ,  2.8428636 , -0.7101018 ,
        0.1384626 , -1.0673387 , -3.370139  , -0.94350225, -1.4588494 ,
       -3.331888  , -1.717582  ,  1.3883634 ,  0.5459598 ,  2.0789754 ,
        1.7518582 ,  1.2627064 , -1.3084236 , -1.8821621 , -1.4310218 ,
       -3.1434245 , -0.14035897, -2.0289671 , -0.14818999, -3.0081518 ,
        0.9224236 , -0.54681677, -1.4460645 , -2.0324495 ,  2.25

In [27]:
model_comment_cbow.similarity('merasakan','pembangunan')

C:\Users\Rumondang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.34116235

In [28]:
model_comment_cbow.most_similar('merasakan')[:5]

C:\Users\Rumondang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('rasakan', 0.7191861867904663),
 ('dirasakan', 0.7115248441696167),
 ('melihat', 0.6228532195091248),
 ('menikmati', 0.6043096780776978),
 ('alami', 0.5748023390769958)]

In [29]:
#train model comment cbow
model_comment_cbow.train(df_tbk_comment,total_examples=len(df_tbk_comment),epochs=10)

(76376780, 83327500)

In [30]:
#vocab comment cbow model
vocab_comment = model_comment_cbow.wv.vocab

In [31]:
model_comment_cbow.wv.most_similar('merasakan')[:5]

[('rasakan', 0.7150262594223022),
 ('dirasakan', 0.6935787200927734),
 ('ngerasain', 0.6019014716148376),
 ('menikmati', 0.5968118906021118),
 ('melihat', 0.5777407884597778)]

In [32]:
model_comment_cbow.wv.most_similar('mmg')[:5]

[('mmng', 0.7512494921684265),
 ('memang', 0.7051973938941956),
 ('emg', 0.6886371970176697),
 ('mgkn', 0.5894177556037903),
 ('emng', 0.5767183303833008)]

In [34]:
max_comment_size = len(model_comment_cbow.wv.syn0)

C:\Users\Rumondang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [35]:
print(max_comment_size)

348075


untuk bagian bahasa baku

In [36]:
vocab_words=list(df_cwords_kata)
vocab_sentence=list(df_csentence_kalimat)

In [37]:
#filter kata baku dan non baku
non_KBBI = []
dict_KBBI = {}

for kalimat in df_tbk_comment:
  for kata in kalimat:
    if kata in vocab_words:
      dict_KBBI[kata] = kata
    else:
      non_KBBI.append(kata)

In [98]:
kata_non_kbbi = pd.Series(non_KBBI)
kata_non_kbbi.head()

0    wkwkwk
1       dpr
2    mantab
3        br
4        kl
dtype: object

In [99]:
df = pd.DataFrame(pd.np.empty((0, 4))) 
kata_non_kbbi = pd.read_csv('Map Kata Non KBBI.csv')

In [100]:
kata_non_kbbi.columns = ['kata tidak baku']
list_kata_non_kbbi = list(kata_non_kbbi['kata tidak baku']) 
print(len(list_kata_non_kbbi))

2159291


In [56]:
df_new_konkat_nonkbbi["non-KBBI"].value_counts().tail(10000)

ambisiambisiambisi                        1
manulakayak                               1
inipolisikan                              1
sajabiasa                                 1
honerer                                   1
wakilnyamereka                            1
demokasi                                  1
culasmalingnipubocorbubardunguretorika    1
korutsilahkan                             1
jubahrumah                                1
menghiburtp                               1
rabl8ndungilah                            1
1bulanbayar                               1
bosannyinyirmuluayoadugagasanprogram      1
fahro                                     1
yaaaagk                                   1
basr                                      1
barbek                                    1
gelepartruz                               1
koalisu                                   1
didalamduitnya                            1
xxxvvii                                   1
bungkau                         

In [47]:
df_new_konkat = pd.DataFrame.from_dict(dict_KBBI,orient="index")

In [202]:
df_new_konkat

,level_0,index,Non-KBBI,KBBI
0,0,0,gak,gak
1,1,1,sekalian,sekalian
2,2,2,pak,pak
3,3,3,selawat,selawat
4,4,4,ke,ke
5,5,5,kan,kan
6,6,6,banyak,banyak
7,7,7,koruptor,koruptor
8,8,8,disana,disana
9,9,9,ini,ini


In [105]:
df_slang = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')

In [106]:
df_slang

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0
5,eeeehhhh,eh,1,Eh ada @ghessawarsana . Eeeehhhh,elongasi,0,0
6,kata2nyaaa,kata-katanya,0,Kata2nyaaa ?,reduplikasi,elongasi,0
7,hallo,halo,1,Hallo kakak tulus,elongasi,0,0
8,kaka,kakak,1,Ngefans banget sama kaka? sukses selalu ka? @t...,zeroisasi,0,0
9,ka,kak,1,Ngefans banget sama kaka? sukses selalu ka? @t...,zeroisasi,0,0


In [161]:
len(kata_formal)

df = pd.DataFrame(list(zip(kata_non_formal, kata_formal ,category_kata, vector_sebelum, vector_setelah)), columns =['Kata awal', 'Kata KBBI', 'Kategori', 'Vector Sebelum', 'Vector Setelah']) 

In [162]:
slang_incomment = df_slang['slang'] == 'br'
df_slang['formal'][slang_incomment].head()

966     baru
2516    baru
2525    baru
3834    baru
7705    baru
Name: formal, dtype: object

In [227]:
print("Melakukan Processing Data")
import nltk
komentar_baru = []
Kata_baru = []
per_kata = 0
for comment in df_tbk_comment:
    komentar = []
    for bag_comment in comment:
        if bag_comment in df_cwords:
            checkslang = df_slang[df_slang['slang'] == bag_comment]
            if len(checkslang) != 0:
                words = checkslang.formal.mode()[0]
                per_kata = per_kata + 1
                komentar_baru.append(words)
            else:
                per_kata = per_kata + 1
                komentar_baru.append(bag_comment)
        else:
            new_comment = False
            checkslang = df_slang[df_slang['slang'] == bag_comment]
            if len(checkslang) != 0:
                words = checkslang.formal.mode()[0]
                per_kata = per_kata + 1
                komentar_baru.append(words)
                new_comment = True
            else:
                dbk_katas = []
                for slang in df_slang['slang']:
                    dbk_kata = nltk.edit_distance(bag_comment, slang)
                    dbk_katas.append(dbk_kata)
                min_dbk = dbk_katas.index(min(dbk_katas))
                value = str(df_slang.formal.loc[min_dbk])
                per_kata1 = per_kata + 1
                komentar_baru.append(value)
                Kata_baru.append(bag_comment + " = " + value)
                new_comment = True
            if new_comment ==  False:
                komentar_baru.append(bag_comment)
                Kata_baru.append(bag_comment + " = " + "Tidak ada")

In [240]:
print(Kata_baru).head

['wkwkwk = wow', 'sekalian = sekalian', 'pak = enggak', 'selawat = selamat', 'ke = kak', 'dpr = dari', 'kan = kak', 'banyak = banyak', 'koruptor = kostum', 'disana = disana', 'mantab = mantab', 'ini = begini', 'dakwah = dah', 'sejati = sehat-sehat', 'tempat = tempat', 'umum = lucu', 'mah = ya', 'biasa = biasa-biasa', 'ini = begini', 'luar = keluar', 'biasa = biasa-biasa', 'salut = selalu', 'buat = dibuat', 'gus = enggak bisa', 'miftah = meminta', 'nyinyir = menyindir', 'pernah = pernah', 'lihat = lihat', 'dan = dimana', 'baca = baca-baca', 'mengetahui = mengerti', 'sejarah = terjawab', 'dakwah = dah', 'walisongo = maling', 'sunan = ulang', 'kali = kali', 'jagasmua = bagaimana', 'ada = saja', 'cara = percaya', 'demi = dimana', 'kebaikan = kebanyakan', 'asal = hapal', 'ajk = saja', 'cara = percaya', 'lakukan = melakukan', 'porsi = orisinal', 'dan = dimana', 'sikon = semakin', 'tepat = tempat', 'ada = saja', 'adabnya = apa-apanya', 'sholawat = selamat', 'lebih = lebih', 'baik = baik', 'aj

AttributeError: 'NoneType' object has no attribute 'head'